In this notebook we will prototype an implementation of plmcmc using a few adaptive Metropolis-Hastings routines available for R

In [1]:
source("../power_law_aux.r")
library(poweRlaw)
library(rstan)
library(MHadaptive)

Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.18.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
Loading required package: MASS


First, we will define the unnormalised likelihood, a function to compute the normalising constant $\kappa(\boldsymbol\theta, \boldsymbol\phi)$, the priors and the target posterior:

In [2]:
unnorm_lik <- function(x, a, m, p0, p1, p2){
  if(x < m) return(0)
  return(
    x^-a * (1 - exp(-p0 - p1 * (x-1) - p2* (x-1)^2 ))
  )
}
unnorm_lik <- Vectorize(unnorm_lik)
#
get_norm_const <- function(a, m, p0, p1, p2){
  integrate(function(x) unnorm_lik(x, a, m, p0, p1, p2), m, Inf)
}
#
get_transformed_pars <- function(pars){
  K <- length(pars)
  tpars <- rep(NA, K)
  tpars[1] <- exp(pars[1]) + 1
  tpars[2:K] <- exp(pars[2:K])
  return(tpars)
}
#
likelihood <- function(pars, data, m){
  tpars <- get_transformed_pars(pars)
  const <- get_norm_const(a = tpars[1], m = m,  p0 = tpars[2],
                          p1 =  tpars[3], p2 = tpars[4])$value
  lik <- sum(
    log(
      sapply(1:length(data$v), function(i){
        data$fs[i] * unnorm_lik(x = data$v[i],
                                a = tpars[1], m = m,
                                p0 = tpars[2], p1 =  tpars[3],
                                p2 =  tpars[4])/const
      }
      )
    )
  )
  return(lik)
}
##
prior <- function(pars){
  tpars <- get_transformed_pars(pars)
  dunif(tpars[1], 1, 8, log = TRUE) +
  sum(dexp(tpars[2:4], rate = 1/3, log = TRUE))  
}
#
target <- function(pars, data, m, verbose = FALSE){
  if(verbose) cat("pars:", get_transformed_pars(pars), "\n")
  likelihood(pars, data, m) + prior(pars)
}

Data and initial values for the parameters:

In [3]:
data("moby")
the.data <- moby
dt <- compress_data(the.data)
pp <- c(
  log(2.2-1), ## alpha
  log(0.2), ## phi_0
  log(.5), ## phi_1
  log(1.2) ## phi_2
)

In [4]:
## Testing
likelihood(pars = pp, data = dt, m = 1)
prior(pp)
target(pars = pp, data = dt, m = 1)

[1] -2595.266

[1] -5.87508

[1] -2601.141

In [5]:
neglike <- function(alpha_raw, phi0_raw, phi1_raw, phi2_raw){
    lesp <- c(alpha_raw, phi0_raw, phi1_raw, phi2_raw)
    res <-  -likelihood(pars = lesp, data = dt, m = 1)
    return(res)
}

In [6]:
neglike(pp[1], pp[2], pp[3], pp[4])

[1] 2595.266

In [7]:
stats4::mle(neglike,
    start = list(alpha_raw = pp[1], phi0_raw = pp[2], phi1_raw = pp[3], phi2_raw = pp[4]),
    method = "BFGS")

ERROR: Error in integrate(function(x) unnorm_lik(x, a, m, p0, p1, p2), m, Inf): maximum number of subdivisions reached


In [8]:
get_transformed_pars(c(alpha_raw = -1293.57546347681, phi0_raw = -26.5732515285779, 
                       phi1_raw = 35.9520153823283, phi2_raw = -84.4073055956611))

[1] 1.000000e+00 2.879940e-12 4.109244e+15 2.199748e-37

First in line is [**MHadaptive**](https://cran.r-project.org/package=MHadaptive):

In [9]:
plMCMC <- Metro_Hastings(li_func = target, pars = pp,
                         iterations = 5E4, 
                       par_names = c('alpha','phi0','phi1', 'phi2'),
                       data = dt,
                       burn_in = 5000,
                       m = 1, 
                       verbose = FALSE)

ERROR: Error in solve.default(-fit$hessian): Lapack routine dgesv: system is exactly singular: U[3,3] = 0


In [ ]:
# plMCMC$trace <- t(apply(plMCMC$trace, 1, get_transformed_pars))  

Next in line is the [**mcmc**](https://cran.r-project.org/web/packages/mcmc/mcmc.pdf) package:

In [10]:
library(mcmc)
GeyerMCMC <- mcmc::metrop(target, data = dt, m = 1, init = pp, nbatch = 1E4)

ERROR: Error in integrate(function(x) unnorm_lik(x, a, m, p0, p1, p2), m, Inf): the integral is probably divergent


Timing stopped at: 238.2 0.179 239.3


In [ ]:
get_transformed_pars(c(0.370476820884172, -12.9894667495059, -17.2603232286185, -13.6337881364319))